# Warning : 
# Do "File -> Save a copy in Drive" before you start modifying the notebook, otherwise your modifications will not be saved.

# 0) Loading MNIST data

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
from torchvision import datasets, transforms
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
# Load MNIST dataset
bsize = 100
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])
train_dataset = datasets.MNIST('data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=bsize, shuffle=True)#, **kwargs)
test_loader = DataLoader(test_dataset, batch_size=bsize)#, **kwargs)

# Visualize some images
images, labels = next(iter(train_loader))
print(images.shape)
print(labels.shape)
fig, axes = plt.subplots(nrows=4, ncols=4)
for i, (image, label) in enumerate(zip(images, labels)):
    if i >= 16:
        break
    axes[i // 4][i % 4].imshow(images[i][0], cmap='gray')
    axes[i // 4][i % 4].set_title(f"{label}")
    axes[i // 4][i % 4].set_xticks([])
    axes[i // 4][i % 4].set_yticks([])
fig.set_size_inches(8, 8)
fig.tight_layout()

# 1) Classification Models

## 1.A) Let us define a Logistic regression model

In [ ]:
class LR(nn.Module):
  def __init__(self):
    super(LR, self).__init__()
    # YOUR CODE

  def forward(self, x):
    # YOUR CODE

In [ ]:
lr = LR().to(device)
print(lr)

pred = lr(images.to(device))
print(pred.shape)

## Lets train the LR model!

In [ ]:
optimizer = torch.optim.SGD(lr.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()
nbepochs = 10

for e in range(nbepochs):
  total_loss, correct = 0.0, 0.0
  for (images, labels) in train_loader: 
    # ZERO GRAD
    optimizer.zero_grad()
    
    im, lab = images.to(device), labels.to(device)
    # FORWARD : YOUR CODE
    #pred = 
    # LOSS COMPUTATION : YOUR CODE
    #loss = 
    total_loss += loss
    correct += (pred.argmax(-1) == lab).sum()
    
    # BACKWARD + GRADIENT STEP : YOUR CODE


  print(f"[Epoch {e + 1:2d}] loss: {total_loss/ len(train_dataset):.2E} accuracy_train: {correct / len(train_dataset):.2%}")


In [ ]:
def evalTest(testloader, net):
  total_loss, nbSamples, correct=0.0, 0.0, 0.0

  for (images, labels) in testloader: 
    optimizer.zero_grad()
    
    im, lab = images.to(device), labels.to(device)
    pred = net(im)

    loss = criterion(pred, lab)
    total_loss += loss
    correct += (pred.argmax(-1) == lab).sum()
    nbSamples += images.shape[0]

  acc = correct / nbSamples*100.0
  return acc, total_loss / nbSamples

In [ ]:
accT= evalTest(test_loader, lr)[0]
print(f"Test accuracy={accT:.2f}%")

## 1.B) Let's now define a MLP with a single hidden layer of size 100

In [ ]:
class SimpleMLP(nn.Module):
  def __init__(self):
    super(SimpleMLP, self).__init__()
    # YOUR CODE

  def forward(self, x):
    # YOUR CODE


## Lets train the MLP model!

In [ ]:
optimizer = torch.optim.SGD(mlp.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()
nbepochs = 20

for e in range(nbepochs):
  total_loss, correct = 0.0, 0.0
  for (images, labels) in train_loader: 
    # BATCH LOOP: YOUR CODE 


In [ ]:
accT= evalTest(test_loader, mlp)[0]
print(f"Test accuracy={accT:.2f}%")

## 1.C) Let's now define a ConvNet with a two [Conv + Pool] layers and one hidden layer: 


1.   Conv1: 16 filters of size 5x5, no padding ('valid)'
  - ReLU
  - Pool1: MaxPool2d, stride 2
2.   Conv2: 32 filters of size 5x5, no padding 
  - ReLU
  - Pool1: MaxPool2d, stride 2
3. One fully connected layer of size 100 (flatteing before applying it)
3. One fully connected layer of size 10 (number of classes)


In [ ]:
class LeNet(nn.Module):
  def __init__(self):
      super(LeNet, self).__init__()
      # self.conv1 = 
      # self.maxpool1 = 
      # self.conv2 = 
      # self.maxpool2 = 
      # self.fc1 = 
      # self.fc2 = 

    def forward(self, x):
      # YOUR CODE

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    

In [ ]:
def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
lenet = LeNet().to(device)
print(lenet)

npar = count_parameters(lenet)
print(f"number of parameters in lenet ={npar :d}")

## Lets train the ConvNet!

In [ ]:
Y_testoptimizer = torch.optim.SGD(lenet.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()
nbepochs = 20
# TRAINING LOOP: YOUR CODE 



In [ ]:
accT= evalTest(test_loader, lenet)[0]
print(f"Test accuracy={accT:.2f}%")

# 2) Manifold untangling

## Let's get the full test data 

In [ ]:
import numpy as np 

X_test = np.zeros((10000,784))
Y_test = np.zeros((10000,))

bsize=100

for id, (images, labels) in enumerate(test_loader): 
    X_test[id*bsize:id*bsize+bsize,:] = images.view(-1,784)
    Y_test[id*bsize:id*bsize+bsize] = labels

## 2.A) t-SNE on raw data

In [ ]:
from sklearn.manifold import TSNE

import matplotlib.cm as cm
import numpy as np
from scipy.spatial import ConvexHull
from sklearn.mixture import GaussianMixture
from scipy import linalg
from sklearn.neighbors import NearestNeighbors

# TSNE: creation + call fit_transform to get the 2D projection X_r
#tsne = 
#X_r = 

In [ ]:
def best_ellipses(points, labels):
  # computing best fiiting ellipse for a set of points with asscoiated labels
  gaussians = []
  for i in range(10):
    gaussians.append(GaussianMixture(n_components=1, covariance_type='full').fit(points[labels==i, :]))
  return gaussians

def neighboring_hit(points, labels):
  k = 6
  nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(points)
  distances, indices = nbrs.kneighbors(points)

  txs = 0.0
  txsc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  nppts = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

  for i in range(len(points)):
    tx = 0.0
    for j in range(1,k+1):
      if (labels[indices[i,j]]== labels[i]):
        tx += 1
    tx /= k
    txsc[int(labels[i])] += tx
    nppts[int(labels[i])] += 1
    txs += tx

  for i in range(10):
    txsc[i] /= nppts[i]

  return txs / len(points)

In [ ]:
# computing best fitting ellipses
ellipses = best_ellipses(X_r, labelsT)
# computing nh
nh = neighboring_hit(X_r, labelsT)

In [ ]:
def visualization(points2D, labels, ellipses, nh):
    points2D_c= []
    for i in range(10):
        points2D_c.append(points2D[labels==i, :])
    # Data Visualization
    cmap =cm.tab10
    
    plt.figure(figsize=(8, 4), dpi=200)
    plt.set_cmap(cmap)
    plt.subplots_adjust(hspace=0.4 )
    plt.subplot(121)
    plt.scatter(points2D[:,0], points2D[:,1], c=labels,  s=3,edgecolors='none', cmap=cmap, alpha=1.0)
    plt.colorbar(ticks=range(10))
    plt.title("2D t-SNE - NH="+str(nh*100.0))
    
    vals = [ i/10.0 for i in range(10)]
    
    def plot_results(X, Y_, means, covariances, index, title, color):
        splot = plt.subplot(1, 2, 2)
        for i, (mean, covar) in enumerate(zip(means, covariances)):
            v, w = linalg.eigh(covar)
            v = 2. * np.sqrt(2.) * np.sqrt(v)
            u = w[0] / linalg.norm(w[0])
            # as the DP will not use every component it has access to
            # unless it needs it, we shouldn't plot the redundant
            # components.
            if not np.any(Y_ == i):
              continue
            plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], .8, color=color, alpha = 0.2)

            # Plot an ellipse to show the Gaussian component
            angle = np.arctan(u[1] / u[0])
            angle = 180. * angle / np.pi  # convert to degrees
            ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle, color=color)
            ell.set_clip_box(splot.bbox)
            ell.set_alpha(0.6)
            splot.add_artist(ell)

        plt.title(title)

    for i in range(10):
        plot_results(points2D[labels==i, :], ellipses[i].predict(points2D[labels==i, :]), ellipses[i].means_, ellipses[i].covariances_, 0,"t-SNE fitting ellipses", cmap(vals[i]))

    #plt.savefig(projname+".png", dpi=100)
    plt.show()

In [ ]:
import matplotlib as mpl
visualization(X_r, labelsT, ellipses, nh)

**QUESTION:** what wan you conclude from the t-SNE visualization?

## 2.B) t-SNE on latent space from MLP

## The code below enables to extract a latent vector instead of the output prediction, as follows

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [ ]:
mlp.fc1.register_forward_hook(get_activation('fc1'))

H_test_MLP = np.zeros((10000,100))

bsize=100

for id, (images, labels) in enumerate(test_loader): 
    output = mlp(images.to(device))
    H_test_MLP[id*bsize:id*bsize+bsize,:] = activation['fc1'].cpu().numpy()

In [ ]:
# Compute t-SNE on H_test_MLP
# H_r=

In [ ]:
# computing best fitting ellipses
ellipses = best_ellipses(H_r2, labelsT)
# computing nh
nh = neighboring_hit(H_r2, labelsT)
# t-SNE visualization
visualization(H_r2, labelsT, ellipses , nh)

## 2.C) t-SNE on latent space from ConvNet

In [ ]:
# Do the same for the ConvNet
# Latent 'fc1' feature extraction
# TSNE
# Visualization